# CNNによる画像認識

In [1]:
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

170500096/170498071 [==============================] - 8s 0us/step
(50000, 32, 32, 3) (50000, 1)
(10000, 32, 32, 3) (10000, 1)


use_model.py

In [ ]:
# coding:utf-8
import numpy as np
import scipy.misc
import tensorflow as tf
import pickle as p # python2系はcPickle
from keras.utils import np_utils
from keras.models import Sequential, Model, model_from_json
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.datasets import cifar10

import keras.backend.tensorflow_backend as KTF
import matplotlib.pyplot as plt


in_shape = (32, 32, 3)
cls_num = 3

def build_model():
    model = Sequential()

    model.add(Convolution2D(16, 3, 3, border_mode="same", input_shape=in_shape))
    model.add(Activation("relu"))
    model.add(Convolution2D(16, 3, 3, border_mode="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.5))
    # model.add(Dropout(1))

    model.add(Convolution2D(32, 3, 3, border_mode="same"))
    model.add(Activation("relu"))
    model.add(Convolution2D(32, 3, 3, border_mode="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.5))
    # model.add(Dropout(1))

    model.add(Convolution2D(64, 3, 3, border_mode="same"))
    model.add(Activation("relu"))
    model.add(Convolution2D(64, 3, 3, border_mode="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.5))
    # model.add(Dropout(1))

    model.add(Convolution2D(64, 3, 3, border_mode="same"))
    model.add(Activation("relu"))
    model.add(Convolution2D(64, 3, 3, border_mode="same"))
    model.add(Activation("relu"))
    model.add(MaxPooling2D())
    model.add(Dropout(0.5))
    # model.add(Dropout(1))

    model.add(Convolution2D(128, 3, 3, border_mode="same"))
    model.add(Activation("relu"))
    # model.add(BatchNormalization())
    model.add(Dropout(0.5))
    # model.add(Dropout(1))

    model.add(Flatten())
    model.add(Dense(100))
    model.add(Activation("relu"))

    model.add(Dropout(0.5))
    model.add(Dense(cls_num))
    model.add(Activation('softmax'))

    model.compile(loss="categorical_crossentropy",
        metrics   = ["accuracy"],
        optimizer = "adam"
    )
    return model

def change3class(exp_val, obj_val):
    new_exp = []
    new_obj = []
    for (x, y) in zip(exp_val, obj_val):
        if y == 3:
            new_exp.append(x)
            new_obj.append(0)
        elif y == 5:
            new_exp.append(x)
            new_obj.append(1)
        elif y == 1:
            new_exp.append(x)
            new_obj.append(2)
        else:
            pass

    new_exp = np.array(new_exp)
    return new_exp, new_obj


if __name__ == "__main__":
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()
    print(X_train.shape, y_train.shape)
    print(X_test.shape, y_test.shape)

    X_train, y_train = change3class(X_train, y_train)
    X_test, y_test = change3class(X_test, y_test)

    y_train = np_utils.to_categorical(y_train)
    y_test  = np_utils.to_categorical(y_test)

    X_train = X_train.astype('float32')
    X_test  = X_test.astype('float32')
    X_train /= 255.0
    X_test /= 255.0

    print("X_train", X_train.shape)
    print("y_train", y_train.shape)
    print("X_test", X_test.shape)
    print("y_test", y_test.shape)

    model = build_model()
    model.fit(X_train, y_train,
        nb_epoch=250,
        batch_size=128,
        validation_data=(X_test, y_test)
    )

    json_string = model.to_json()
    open('test.json', 'w').write(json_string)
    model.save_weights('test.hdf5')

    # evaluate
    score = model.evaluate(X_test, y_test)
    print("test loss", score[0])
    print("test acc",  score[1])

In [ ]:
import keras
import sys, os
import scipy
import scipy.misc
import numpy as np
from keras.models import model_from_json

import json

imsize = (32, 32)
# 下記フォルダには、適当な犬と猫と自動車の画像を入れて下さい。
testpic = "./testpic/"
keras_model = "./test.json"
keras_param = "./test.hdf5"


def load_image(path):
img = scipy.misc.imread(path, mode="RGB")
img = scipy.misc.imresize(img, imsize)
img = img / 255.0
return img

def get_file(dir_path):
"""
testpicディレクトリの配下の画像を1つのリストにして返す
['244573113_thumb.jpg', 'car1.jpg', 'car2.jpg', 'car3.jpg', 'cat1.jpg', 'cat2.jpg', 'cat3.jpg', 'dog1.jpg', 'dog2.jpg', 'dog3.jpg', 'dog4.jpg', 'dog5.jpg', 'dog6.jpg', 'dog7.jpg']
"""
filenames = os.listdir(dir_path)
return filenames

if __name__ == "__main__":

    pic = get_file(testpic)

    model = model_from_json(open(keras_model).read())
    model.load_weights(keras_param)
    model.summary()

    for i in pic:
    print(i) # ファイル名の出力
    img = load_image(testpic + i)
    #vec = model.predict(np.array([img]), batch_size=1)
    prd = model.predict(np.array([img]))
    print(prd)
    prelabel = np.argmax(prd, axis=1)

    # 各画像ファイルに猫ならファイル名+0が、犬ならファイル名+1、乗り物ならファイル名+2のラベルが付いている
    if prelabel == 0:
        print(">>> 猫")
    elif prelabel == 1:
        print(">>> 犬")
    elif prelabel == 2:
        print(">>> 乗り物")

    print("#"*55)

In [ ]:
import pickle
import numpy as np
import os
import scipy.misc
import random
import shutil

cat_dir = "./cat/"
dog_dir = "./dog/"
car_dir = "./car/"

cat_train_image_dir = "./cat_train/"
cat_test_image_dir  = "./cat_test/"

dog_train_image_dir = "./dog_train/"
dog_test_image_dir  = "./dog_test/"

car_train_image_dir = "./car_train/"
car_test_image_dir  = "./car_test/"

in_shape = (120, 120, 3)


def init():
    for i in [cat_train_image_dir, cat_test_image_dir, dog_train_image_dir, dog_test_image_dir, car_train_image_dir, car_test_image_dir]:
        if not os.path.isdir(i):
            os.makedirs(i)

def load_image(path):
    img = scipy.misc.imread(path, mode="RGB")
    if img.shape != in_shape:
        return
    return img

def get_dirfile(dir_path):
    filenames = os.listdir(dir_path)
    return filenames

def load_id_dict(id_dict_path):
    with open(id_dict_path, "rb") as f:
        id_dict = pickle.load(f)
    print(id_dict_path + "をロードしました．")
    return id_dict

def usable(img_files, images_dir):
    usable = []
    for i in img_files:
        img = load_image(images_dir + i)
        if img is None:
            continue
        else:
            usable.append(i)
    return usable

def dataset(usable):
    trainf = random.sample(usable, 800)
    testf = set(usable) - set(trainf)
    testf = list(testf)
    return trainf, testf


if __name__ =="__main__":
    init()

    cat_files = get_dirfile(cat_dir)
    dog_files = get_dirfile(dog_dir)
    car_files = get_dirfile(car_dir)

    cat_usable = usable(cat_files,cat_dir)
    dog_usable = usable(dog_files, dog_dir)
    car_usable = usable(car_files, car_dir)

    print("cat_usable", len(cat_usable))
    print("dog_usable", len(dog_usable))
    print("car_usable", len(car_usable))

    cat_train, cat_test = dataset(cat_usable)
    dog_train, dog_test = dataset(dog_usable)
    car_train, car_test = dataset(car_usable)
    print("cat_train", len(cat_train))
    print("cat_test", len(cat_test))
    print("dog_train", len(dog_train))
    print("dog_test", len(dog_test))
    print("car_train", len(car_train))
    print("car_test", len(car_test))


    for i in cat_train:
        shutil.copy2(cat_dir + i, cat_train_image_dir + i)
    for i in cat_test:
        shutil.copy2(cat_dir + i, cat_test_image_dir + i)

    for i in dog_train:
        shutil.copy2(dog_dir + i, dog_train_image_dir + i)
    for i in dog_test:
        shutil.copy2(dog_dir + i, dog_test_image_dir + i)

    for i in car_train:
        shutil.copy2(car_dir + i, car_train_image_dir + i)
    for i in car_test:
        shutil.copy2(car_dir + i, car_test_image_dir + i)

In [ ]:
# import gpu_config
# gpu_config.set_tensorflow([3])
# import tensorflow.python.ops.tensor_array_ops
import numpy as np
import os
import scipy.misc
import random
import math
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential, Model, model_from_json
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization
from keras.applications.vgg16 import VGG16
import keras.backend.tensorflow_backend as KTF
from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint


cat_train_image_dir = "./cat_train/"
cat_test_image_dir  = "./cat_test/"
dog_train_image_dir = "./dog_train/"
dog_test_image_dir  = "./dog_test/"
car_train_image_dir = "./car_train/"
car_test_image_dir  = "./car_test/"
in_shape = (120, 120, 3)
cls_num  = 3


def build_model():
    model = Sequential()
    model.add(Convolution2D(64, 3, 3, border_mode="same", input_shape=in_shape))
    model.add(Activation("relu"))
    model.add(MaxPooling2D())

    model.add(Convolution2D(32, 3, 3, border_mode="same"))
    model.add(Activation("relu"))

    model.add(MaxPooling2D())

    model.add(Convolution2D(16, 3, 3, border_mode="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D())


    model.add(Flatten())
    model.add(Dense(50))
    model.add(Activation("relu"))
    model.add(Dense(cls_num))
    model.add(Activation('softmax'))

    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")
    return model



def load_image(path):
    img = scipy.misc.imread(path, mode="RGB")
    img = img / 255.0
    return img

def get_file(dir_path):
    filenames = os.listdir(dir_path)
    return filenames

def mk_x(files, images_dir):
    X_train =[]
    for i in files:
        img = load_image(images_dir + i)
        X_train.append(img)
    return X_train

def mk_y(num):
    wrap_label = []
    dog_label = [1,0,0] # 800
    cat_label = [0,1,0] # 800
    car_label = [0,0,1] # 800
    for i in range(num):
        wrap_label.append(dog_label)
    for i in range(num):
        wrap_label.append(cat_label)
    for i in range(num):
        wrap_label.append(car_label)

    print(np.shape(wrap_label))
    return wrap_label


if __name__ == "__main__":
    cat_train = get_file(cat_train_image_dir)
    cat_test  = get_file(cat_test_image_dir)
    dog_train = get_file(dog_train_image_dir)
    dog_test  = get_file(dog_test_image_dir)
    car_train = get_file(car_train_image_dir)
    car_test  = get_file(car_test_image_dir)

    cat_train = mk_x(cat_train, cat_train_image_dir)
    cat_test  = mk_x(cat_test, cat_test_image_dir)
    dog_train = mk_x(dog_train, dog_train_image_dir)
    dog_test  = mk_x(dog_test, dog_test_image_dir)
    car_train = mk_x(car_train, car_train_image_dir)
    car_test  = mk_x(car_test, car_test_image_dir)

    cat_train = np.array(cat_train)
    cat_test  = np.array(cat_test)
    dog_train = np.array(dog_train)
    dog_test  = np.array(dog_test)
    car_train = np.array(car_train)
    car_test  = np.array(car_test)

    X_train = np.concatenate((cat_train, dog_train, car_train))
    X_test  = np.concatenate((cat_test, dog_test, car_test))
    y_train = np.array(mk_y(800))
    y_test  = np.array(mk_y(200))


    print("X_train", X_train.shape)
    print("y_train", y_train.shape)
    print("X_test", X_test.shape)
    print("y_test", y_test.shape)

    model = build_model()
    model.fit(X_train, y_train, nb_epoch=150, batch_size=128, validation_data=(X_test, y_test))

    json_string = model.to_json()
    open('model2.json', 'w').write(json_string)
    model.save_weights('model2.hdf5')

    # evaluate
    score = model.evaluate(X_test, y_test)
    print("test loss", score[0])
    print("test acc", score[1])